In [1]:
import scipy.linalg as spla
import numpy as np
import itertools

from src.reductive_geometry import get_pauli_basis, VectorSpace, comm

$
\newcommand{\SU}{\mathrm{SU}}
\newcommand{\U}{\mathrm{U}}
\newcommand{\Stab}{\mathrm{Stab}}
\newcommand{\SO}{\mathrm{SO}}
\newcommand{\Sp}{\mathrm{Sp}}
\newcommand{\su}{\mathfrak{su}}
\newcommand{\so}{\mathfrak{so}}
\newcommand{\gl}{\mathfrak{gl}}
\renewcommand{\sp}{\mathfrak{sp}}
\renewcommand{\gg}{\mathfrak{g}}
\newcommand{\uu}{\mathfrak{u}}
\newcommand{\kk}{\mathfrak{k}}
\newcommand{\mm}{\mathfrak{m}}
\newcommand{\oo}{\mathfrak{o}}
\newcommand{\hh}{\mathfrak{h}}
\renewcommand{\tt}{\mathfrak{t}}
\newcommand{\Span}{\mathrm{span}}
$

In [2]:
def format_coeffs(coeffs, names):
    assert len(coeffs)==len(names)
    str_rep = ""
    for i, c_i in enumerate(coeffs):
        if not np.isclose(c_i,0.):
            str_rep = str_rep + f"{c_i:1.5f}*{names[i]} + "
    return str_rep[:-2]

In [ ]:
# Different homogeneous space $G/K$

In this notebook, we will explore how to find the tangent space parameterization $T_{G/K}\:gK \cong \mathfrak{m}$ for a given $G$ and $K$ and associated Lie algebras $\mathfrak{g}$ and $\mathfrak{k}$.

In [3]:
su2_dict = get_pauli_basis(1)
su4_dict = get_pauli_basis(2)
su2_names = list(su2_dict.keys())
su4_names = list(su4_dict.keys())
su2_vspace = VectorSpace(np.stack(list(su2_dict.values())))
su4_vspace = VectorSpace(np.stack(list(su4_dict.values())))

In [4]:
so4 = ['IY', 'XY', 'YI', 'YX', 'YZ', 'ZY']
so4_dict = dict(zip(so4, [su4_dict[p] for p in so4]))    
so4_names = list(so4_dict.keys())
so4_vspace = VectorSpace(np.stack(list(so4_dict.values())))

## 1.) $\SU(2)/\U(1)$

By inspection, we quickly find

$$
\begin{align}
\gg &= \Span\{X,Y,Z\} \\
\kk &= \Span\{Z\}\\
\mm &= \Span\{X,Y\}\\
\end{align}
$$
clearly the center is $\mathfrak{z}(\kk) = \Span\{Z\}$ and $\gg^\kk = 0$.

In [5]:
g = su2_vspace.basis.copy()
k = su2_dict["Z"]
m = [su2_dict["X"], su2_dict["Y"]]

## 2.) $\SU(4)/\U(2)$ (spin 1/2)

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} 
\end{align}
$$

It's clear that

$$
\begin{align}
    \kk &= i\,\Span\{XI+IX, YI+IY, ZI + IZ\}
\end{align}
$$

satisfies the commutation relations of $\su(2)$. We now have a trivial center $\mathfrak{z}(\kk)=0$. To find the space $\mm$, we use a nullspace methodx to find the elements of $\gg$ that are orthogonal to $\kk$. 

In [6]:
I = np.eye(2, dtype=complex)
g = VectorSpace(su4_vspace.basis)
# Create the su(2) basis within su(4)
k = np.stack([np.kron(su2_dict['X'], I) + np.kron(I, su2_dict['X']),
                      np.kron(su2_dict['Y'], I) + np.kron(I, su2_dict['Y']),
                      np.kron(su2_dict['Z'], I) + np.kron(I, su2_dict['Z'])]) / 2
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))
    

Dimension of m: 12
-0.70711*IX + 0.70711*XI 
1.00000*XX 
1.00000*XY 
1.00000*XZ 
-0.70711*IY + 0.70711*YI 
1.00000*YX 
1.00000*YY 
1.00000*YZ 
-0.70711*IZ + 0.70711*ZI 
1.00000*ZX 
1.00000*ZY 
1.00000*ZZ 


Hence

$$
\begin{align}
 \mm=i\,\Span\{XX,XY,XZ,YX,YY,YZ, ZX,ZY,ZZ, XI-IX, YI-IY, ZI - IZ\}
\end{align}
$$

By the the Schur-Weyl duality, we find that $\gg_o^\kk = \mathrm{SWAP}$:

In [7]:
swap = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]])
# Swap commutes with the full basis.
for g_i in g.basis:
    assert np.allclose(comm(swap,g_i), np.zeros_like(swap))
    

## 3a.) $\SU(4) / (I_2 \times \SU(2))$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0_2\oplus X, 0_2\oplus Y,0_2\oplus Z\}
\end{align}
$$
where $0_2$ is a zero matrix of size $2\times 2$. The center is trivial $\mathfrak{z}(\kk)=0$

In [8]:
I = np.eye(2, dtype=complex)
g = VectorSpace(su4_vspace.basis)
k = []
for term in su2_vspace.basis:
    k.append(np.block([[np.zeros((2,2)), np.zeros((2,2))],
                       [np.zeros((2,2)), term]]))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))

Dimension of m: 12
1.00000*XI 
1.00000*XX 
1.00000*XY 
1.00000*XZ 
1.00000*YI 
1.00000*YX 
1.00000*YY 
1.00000*YZ 
1.00000*ZI 
0.70711*IX + 0.70711*ZX 
0.70711*IY + 0.70711*ZY 
0.70711*IZ + 0.70711*ZZ 


Hence

$$
\begin{align}
     \mm=i\,\Span\{XI, XX ,XY ,XZ ,YI ,YX ,YY ,YZ ,ZI ,IX + ZX ,IY + ZY ,IZ + ZZ \}
\end{align}
$$
**TODO:** What is $\gg^\kk_o$?

## 3b.) $\SO(4) / (I_2 \times \SO(2))$

The basis for $\so(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0_2\oplus Y\}
\end{align}
$$
where $0_2$ is a zero matrix of size $2\times 2$. The center is trivial $\mathfrak{z}(\kk)=0$

In [16]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
k = []
k.append(np.block([[np.zeros((2,2)), np.zeros((2,2))],
                   [np.zeros((2,2)), su2_dict['Y']]]))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))
    print(np.allclose(np.zeros((4,4)), comm(m_i,k[0])))

Dimension of m: 5
1.00000*XY 
False
1.00000*YI 
False
1.00000*YX 
False
1.00000*YZ 
False
0.70711*IY + 0.70711*ZY 
True


Hence

$$
\begin{align}
     \mm=i\,\Span\{XY,YI ,YX ,YZ, IY + ZY\}
\end{align}
$$
**TODO:** What is $\gg^\kk_o$?

## 3c.) $\SO(4) / (1 \times \SO(2) \times 1)$

The basis for $\so(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0\oplus Y\oplus0\}
\end{align}
$$

The center is trivial $\mathfrak{z}(\kk)=0$

In [19]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
k = []
k.append(np.block([[0,0,0,0],[0,0,-1j,0],[0,1j,0,0],[0,0,0,0]]))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))
    print(np.allclose(np.zeros((4,4)), comm(m_i,k[0])))

Dimension of m: 5
0.70711*IY + 0.50000*XY + 0.50000*YX 
False
1.00000*YI 
False
-0.70711*IY + 0.50000*XY + 0.50000*YX 
False
1.00000*YZ 
False
1.00000*ZY 
False


Hence

$$
\begin{align}
     \mm=i\,\Span\{\sqrt{2}IY+ XY+YX,-\sqrt{2}IY+ XY+YX, YI,YZ, ZY \}
\end{align}
$$
**TODO:** What is $\gg^\kk_o$?

## 4.) $\SU(4)/\SU(2)$ (spin 3/2)

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\:\Span\{S_x, S_y, S_z\}
\end{align}
$$

with 

$$
\begin{align}
S_x &= \begin{bmatrix}
    0 & \sqrt{3} & 0 & 0 \\
    \sqrt{3} & 0 & 2 & 0 \\
    0 & 2 & 0 & \sqrt{3} \\
    0 & 0 & \sqrt{3} & 0 \\
    \end{bmatrix},\quad S_y = \begin{bmatrix}
    0 & -i\sqrt{3} & 0 & 0 \\
    i\sqrt{3} & 0 & -2i & 0 \\
    0 & 2i & 0 & -i\sqrt{3} \\
    0 & 0 & i\sqrt{3} & 0 \\
    \end{bmatrix},\quad S_z = \begin{bmatrix}
    3 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & -3 \\
    \end{bmatrix},
\end{align}
$$

we find $\mm$ via the null space method.

In [10]:
g = VectorSpace(su4_vspace.basis)
Sx = np.array([[0, np.sqrt(3), 0, 0],
               [np.sqrt(3), 0, 2, 0],
               [0, 2, 0, np.sqrt(3)],
               [0, 0, np.sqrt(3), 0]]) / 2
Sy = np.array([[0, -1j * np.sqrt(3), 0, 0],
               [1j * np.sqrt(3), 0, -2j, 0],
               [0, 2j, 0, -1j * np.sqrt(3)],
               [0, 0, 1j * np.sqrt(3), 0]]) / 2
Sz = np.array([[3, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, -3]]) / 2
k = np.stack([Sx, Sy, Sz])

subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))
    

Dimension of m: 12
1.00000*XI 
-0.44721*IX + 0.88730*XX + -0.11270*YY 
0.44721*IY + 0.88730*XY + 0.11270*YX 
1.00000*XZ 
1.00000*YI 
-0.44721*IY + 0.11270*XY + 0.88730*YX 
-0.44721*IX + -0.11270*XX + 0.88730*YY 
1.00000*YZ 
-0.89443*IZ + 0.44721*ZI 
1.00000*ZX 
1.00000*ZY 
1.00000*ZZ 


Hence,

$$
\begin{align}
 \mm = &i\,\Span\{XI 
-aIX + bXX + -cYY ,
aIY + bXY + cYX ,
XZ,YI ,
-aIY + cXY + bYX ,\\
&-aIX + -cXX + bYY ,
YZ ,
-dIZ + aZI, 
ZX ,ZY ,ZZ \}
\end{align}
$$

With the coefficients $a= \frac{1}{\sqrt{5}}$, $b=\frac{5+\sqrt{15}}{10}$, $c = \frac{5-\sqrt{15}}{10}$ and $d=\frac{2}{\sqrt{5}}$ (by plugging in the above floating point numbers into Wolfram Alpha). We find $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

## 5.) $\SU(4)/\Sp(2)$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= \Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

Since $\SU(4)/\Sp(2)$ is a symmetric space, we can find a basis for $\Sp(2)$ by defining an involution $\theta:\gg\to\gg$ such that $\kk$ and $\mm$ correspond to the positive and negative eigenspaces of $\theta$, respectively. The involution that results in the split $\su(4) = \mm\oplus\sp(2)$ is 

$$
\begin{align}
    \theta(A) = J \cdot A^* \cdot J^T
\end{align}
$$
where 
$$
\begin{align}
J = \begin{pmatrix}
0 & I_2 \\
-I_2 & 0
\end{pmatrix}
\equiv iYI
\end{align}
$$

In [11]:
# Create a random matrix in SU(4)
coeffs = np.random.randn(15)
mat = 1j * su4_vspace.construct_op(coeffs)
g = VectorSpace(su4_vspace.basis)

# Use the involution to split off k
J = np.kron(1j * su2_dict['Y'], np.eye(2)) * np.sqrt(2)
theta_mat = J @ mat.conj() @ J.T

# m is the negative eigenspace
m = 0.5 * (mat - theta_mat)
# k is the positive eigenspace
spn = 0.5 * (mat + theta_mat)

coeffs_m = g.project_coeffs(m).imag
print("m:", format_coeffs(coeffs_m, su4_names))
coeffs_spn = g.project_coeffs(spn).imag
print("k:", format_coeffs(coeffs_spn, su4_names))

m: -0.23174*IX + 1.03084*IZ + 0.99037*XY + 0.50344*YY + -0.35456*ZY 
k: 0.71932*IY + -0.91168*XI + -0.11658*XX + 0.31167*XZ + 0.12762*YI + -0.42667*YX + -1.69243*YZ + 0.83611*ZI + -0.06444*ZX + -0.15624*ZZ 


Hence we find the bases

$$
\begin{align}
\kk &= i\,\Span\{IY ,XI ,XX ,XZ ,YI,YX ,YZ ,ZI ,ZX ,ZZ \}\\
\mm &= i\,\Span\{IX,IZ,XY,YY,ZY \}
\end{align}
$$

with $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

## 6.) $\SU(4) / (\SU(2)\times\SU(2))$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg &= \Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

We see that $\kk$ is given by
$$
\begin{align}
    \kk &= i\:\Span\{XI, YI, ZI, IX, IY, IZ\}\\
\end{align}
$$
hence we can immediately conclude that
$$
\begin{align}
    \mm = i\:\Span\{XX, YY, ZZ, XY, XZ, YX, YZ, ZX, ZY\}\\
\end{align}
$$
which is the orthogonal complement of $\kk$ in $\gg$. Both centers are trivial, $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

## 7.) $\SO(4)/\SU(2)$

A basis for $\gg$ is given by the set of anti-symmetric matrices,

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

It is quickly confirmed that 

$$
\begin{align}
    \kk &= i\:\Span\{K_1, K_2, K_3\}\\
\end{align}
$$

with

$$
\begin{align}
    K_1 &=(YI+IY),\quad K_2=XY + YX,\quad K_3 = YZ + ZY
\end{align}
$$

satisfies the commutation relations of $\su(2)$. Since $\SO(4)/\SU(2)$ is a symmetric space, we could either use an involution to find $\mm$ or construct the orthogonal complement of $\kk$. Here, we do the latter.

In [12]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
# Create the su(2) basis within su(4)
k = np.stack([np.kron(su2_dict['Y'], I) + np.kron(I, su2_dict['Y']),
              np.kron(su2_dict['Y'], su2_dict['X']) + np.kron(su2_dict['X'], su2_dict['Y']),
              np.kron(su2_dict['Z'], su2_dict['Y']) + np.kron(su2_dict['Y'], su2_dict['Z'])]) / 2
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))
    

Dimension of m: 3
-0.70711*XY + 0.70711*YX 
0.50000*IY + -0.50000*YI + 0.50000*YZ + -0.50000*ZY 
0.50000*IY + -0.50000*YI + -0.50000*YZ + 0.50000*ZY 


Hence we obtain

$$
\begin{align}
    \mm = i\:\Span\{YX-XY, IY-YI + YZ-ZY, IY-YI + ZY- YZ\}
\end{align}
$$

Both centers are trivial, $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

## 8.) $\SU(4)/\SU(3)$

In [13]:
# Create a random matrix in SU(4)
coeffs = np.random.randn(15)
mat = 1j * su4_vspace.construct_op(coeffs)
g = VectorSpace(su4_vspace.basis)

# Use the involution to split off k
p,q = (1,3)
Ip = np.eye(p)
Iq = np.eye(q)
zero_pq = np.zeros((p, q))
Ipq = np.block([[-Ip, zero_pq],
                [zero_pq.T, Iq]])

theta_mat = Ipq @ mat @ Ipq

# m is the negative eigenspace
m = 0.5 * (mat - theta_mat)
# k is the positive eigenspace
spn = 0.5 * (mat + theta_mat)

coeffs_m = g.project_coeffs(m).imag
print("m:", format_coeffs(coeffs_m, su4_names))
coeffs_spn = g.project_coeffs(spn).imag
print("k:", format_coeffs(coeffs_spn, su4_names))

m: 0.15554*IX + 0.03550*IY + -0.98317*XI + -0.53996*XX + -0.52698*XY + -0.98317*XZ + 0.49071*YI + -0.52698*YX + 0.53996*YY + 0.49071*YZ + 0.15554*ZX + 0.03550*ZY 
k: -0.57138*IX + 0.06849*IY + 0.18249*IZ + 0.39778*XI + 0.63297*XX + -0.37298*XY + -0.39778*XZ + 0.01901*YI + 0.37298*YX + 0.63297*YY + -0.01901*YZ + -0.15495*ZI + 0.57138*ZX + -0.06849*ZY + -0.00344*ZZ 


From which we quickly find

$$
\begin{align}
    \mm &= i\:\Span\{(I+Z)X, X(I+Z),(I+Z)Y, Y(I+Z),XY+YX, XX-YY\}\\
    \kk &= i\:\Span\{(I-Z)X, (I-Z)Y, X(I-Z), Y(I-Z), XY-YX, XX+YY, IZ, ZI, ZZ\}
\end{align}
$$

**TODO:** Is there something in $\gg^\kk_o$?

## 9.) $\SO(4) / \SO(3)$ 

In [14]:
# Create a random matrix in SU(4)
coeffs = np.random.randn(6)
mat = 1j * so4_vspace.construct_op(coeffs)
g = VectorSpace(so4_vspace.basis)

# Use the involution to split off k
p,q = (1,3)
Ip = np.eye(p)
Iq = np.eye(q)
zero_pq = np.zeros((p, q))
Ipq = np.block([[-Ip, zero_pq],
                [zero_pq.T, Iq]])

theta_mat = Ipq @ mat @ Ipq

# m is the negative eigenspace
m = 0.5 * (mat - theta_mat)
# k is the positive eigenspace
spn = 0.5 * (mat + theta_mat)

coeffs_m = g.project_coeffs(m).imag
print("m:", format_coeffs(coeffs_m, so4_names))
coeffs_spn = g.project_coeffs(spn).imag
print("k:", format_coeffs(coeffs_spn, so4_names))

m: 1.66569*IY + -0.98818*XY + -0.36987*YI + -0.98818*YX + -0.36987*YZ + 1.66569*ZY 
k: -0.94913*IY + -0.30188*XY + -0.02979*YI + 0.30188*YX + 0.02979*YZ + 0.94913*ZY 


From which we quickly find

$$
\begin{align}
    \mm &= i\:\Span\{(I+Z)Y, Y(I+Z), XY+YX\}\\
    \kk &= i\:\Span\{(I-Z)Y, Y(I-Z), XY-YX\}
\end{align}
$$

**TODO:** Is there something in $\gg^\kk_o$?

# SWAP gates as quotients

In [29]:
def parameteric_swap(theta):
    return spla.expm(1j* theta*(su4_dict['XX'] + su4_dict['YY']))

print(parameteric_swap(-np.pi/2))

[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]]
